In [49]:
import os 
import pandas as pd 
import random 
import numpy as np 

covid_images = os.listdir('./Dataset_COVID/COVID/Images')
random.shuffle(covid_images)
normal_images = os.listdir('./Dataset_COVID/Normal/Images')
random.shuffle(normal_images)

print(covid_images.__len__())

all_images = normal_images[:3000] + covid_images[:3000]
random.shuffle(all_images)
validation_images = normal_images[3000:3616] + covid_images[3000:]
random.shuffle(validation_images)
all_images.__len__()

1000


2000

In [2]:
from keras.models import Model 
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dense
from keras.layers import UpSampling2D, Add, Multiply, concatenate, Lambda
from keras.layers import Dropout, Layer, Flatten, MaxPool2D, multiply
from keras.callbacks import ModelCheckpoint

2022-10-18 15:03:23.933812: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
SIZE = 256
class EncoderBlock(Layer):

    def __init__(self, filters, rate, pooling=True, **kwargs):
        super(EncoderBlock, self).__init__(**kwargs)

        self.filters = filters
        self.rate = rate
        self.pooling = pooling

        self.c1 = Conv2D(filters, kernel_size=3, padding='same', activation='relu', kernel_initializer='he_normal')
        self.drop = Dropout(rate)
        self.c2 = Conv2D(filters, kernel_size=3, padding='same', activation='relu', kernel_initializer='he_normal')
        self.pool = MaxPool2D()

    def call(self, X):
        x = self.c1(X)
        x = self.drop(x)
        x = self.c2(x)
        if self.pooling:
            y = self.pool(x)
            return y, x
        else:
            return x

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            'filters':self.filters,
            'rate':self.rate,
            'pooling':self.pooling
            }
    
class DecoderBlock(Layer):

    def __init__(self, filters, rate, **kwargs):
        super(DecoderBlock, self).__init__(**kwargs)

        self.filters = filters
        self.rate = rate

        self.up = UpSampling2D()
        self.net = EncoderBlock(filters, rate, pooling=False)

    def call(self, X):
        X, skip_X = X
        x = self.up(X)
        x = concatenate([x, skip_X])
        x = self.net(x)
        return x

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            'filters':self.filters,
            'rate':self.rate
            }
    
class AttentionGate(Layer):

    def __init__(self, filters, **kwargs):
        super(AttentionGate, self).__init__(**kwargs)

        self.filters = filters
        self.normal = Conv2D(filters, kernel_size=3, padding='same', kernel_initializer='he_normal', activation='relu')
        self.down = Conv2D(filters, kernel_size=3, strides=2, padding='same', kernel_initializer='he_normal', activation='relu')
        self.add = Add()
        self.resample = UpSampling2D()
        self.learn = Conv2D(1,kernel_size=1, padding='same', activation='sigmoid') 
        self.adapt = Multiply()

    def call(self, X):
        X, skip_X = X
        x = self.normal(X)
        skip = self.down(skip_X)
        x = self.add([x, skip])
        x = self.learn(x)
        x = self.resample(x)
        f = self.adapt([x,skip_X])
        return f

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            'filters':self.filters,
            }
    
    
    

In [4]:
input_layer = Input(shape=(SIZE, SIZE, 3), name="InputLayer")

# Encoder Network
p1, c1 = EncoderBlock(32, 0.1, name="Encoder1")(input_layer)
p2, c2 = EncoderBlock(64, 0.1, name="Encoder2")(p1)
p3, c3 = EncoderBlock(128, 0.2, name="Encoder3")(p2)
p4, c4 = EncoderBlock(256, 0.2, name="Encoder4")(p3)

# Encoding
c5 = EncoderBlock(512, 0.3, pooling=False, name="EncodingLayer")(p4)

# Attention + Decoder
a1 = AttentionGate(256, name="Attention1")([c5, c4])
d1 = DecoderBlock(256,0.2, name="Decoder1")([c5, a1])

a2 = AttentionGate(128, name="Attention2")([d1, c3])
d2 = DecoderBlock(128,0.2, name="Decoder2")([d1, a2])

a3 = AttentionGate(64, name="Attention3")([d2, c2])
d3 = DecoderBlock(64,0.1, name="Decoder3")([d2, a3])

a4 = AttentionGate(32, name="Attention4")([d3, c1])
d4 = DecoderBlock(32,0.1, name="Decoder4")([d3, a4])

# Output
output_layer = Conv2D(1, kernel_size=3, padding='same', activation='sigmoid')(d4)

# Model
model = Model(inputs=[input_layer], outputs=[output_layer])
model.summary()

2022-10-18 14:40:50.117245: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 InputLayer (InputLayer)        [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 Encoder1 (EncoderBlock)        ((None, 128, 128, 3  10144       ['InputLayer[0][0]']             
                                2),                                                               
                                 (None, 256, 256, 3                                               
                                2))                                                               
                                                                                              

In [5]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [7]:
from tensorflow import keras
model = keras.models.load_model('./Dataset_COVID/segmentation_model')

2022-10-18 17:47:47.245021: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 InputLayer (InputLayer)        [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 Encoder1 (EncoderBlock)        ((None, 128, 128, 3  10144       ['InputLayer[0][0]']             
                                2),                                                               
                                 (None, 256, 256, 3                                               
                                2))                                                               
                                                                                            

In [37]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
import matplotlib.pyplot as plt 
import numpy as np 
import cv2

base_dir = './Dataset_COVID/'
target_dir = './Dataset_COVID/Segmented_Images/'

for image in all_images:
    x = np.zeros((1,) + (256, 256) + (3,), dtype="float32")
    target = "Normal" if image[:5] == "Norma" else "COVID"
    img_path = base_dir + f'{target}/images/'+ image
    image_ = np.array(load_img(img_path, target_size=(256,256)))
    x[0] = image_/255.
    mask = np.array(model.predict(x, batch_size=1))*255
    mask = mask.astype(int)
    processed_image = np.bitwise_and(image_, mask[0])
    cv2.imwrite(target_dir+image, processed_image)
    

1/1 [==============================] - 0s 280ms/step


1/1 [==============================] - 0s 283ms/step


1/1 [==============================] - 0s 286ms/step


1/1 [==============================] - 0s 291ms/step


1/1 [==============================] - 0s 291ms/step


...

1/1 [==============================] - 0s 304ms/step


1/1 [==============================] - 0s 298ms/step


1/1 [==============================] - 0s 306ms/step


1/1 [==============================] - 0s 287ms/step


1/1 [==============================] - 0s 332ms/step


In [38]:
len(all_images)

2000

In [48]:
for i in validation_images:
    print(i)